<a href="https://colab.research.google.com/github/poojamahajan0712/Data-Science-Portfolio/blob/main/NLP/Machine%20Translation/MachineTranslation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install "tensorflow-text>=2.11"
!pip install einops

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 14.1 MB/s eta 0:00:00


In [3]:
import tensorflow as tf
import tensorflow_text as tf_text

import pathlib

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from collections import Counter

In [5]:
dataset_path = "/content/drive/MyDrive/Machine_Translation/eng_-french/eng_-french.csv"
df1 = pd.read_csv(dataset_path)
df1.tail()

,English words/sentences,French words/sentences
175616,"Top-down economics never works, said Obama. ""T...","« L'économie en partant du haut vers le bas, ç..."
175617,A carbon footprint is the amount of carbon dio...,Une empreinte carbone est la somme de pollutio...
175618,Death is something that we're often discourage...,La mort est une chose qu'on nous décourage sou...
175619,Since there are usually multiple websites on a...,Puisqu'il y a de multiples sites web sur chaqu...
175620,If someone who doesn't know your background sa...,Si quelqu'un qui ne connaît pas vos antécédent...


In [16]:
df1.head()

,English words/sentences,French words/sentences
0,Hi.,Salut!
1,Run!,Cours !
2,Run!,Courez !
3,Who?,Qui ?
4,Wow!,Ça alors !


In [13]:
context = np.array([v[0] for k,v in df1.iterrows()])
target = np.array([v[1] for k,v in df1.iterrows()])


<ipython-input-13-feb71b15fb94>:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  context = np.array([v[0] for k,v in df1.iterrows()])
<ipython-input-13-feb71b15fb94>:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  target = np.array([v[1] for k,v in df1.iterrows()])


In [25]:
context[1],target[1] ## space before ! is encoded as \u202f

('Run!', 'Cours\u202f!')

In [26]:
context[3],target[3]

('Who?', 'Qui ?')

In [27]:
context[4],target[4]

('Wow!', 'Ça alors\u202f!')

In [28]:
context[2],target[2]

('Run!', 'Courez\u202f!')

In [29]:
np.random.uniform(size=(len(target),))

array([0.88918454, 0.11297055, 0.26440285, ..., 0.21573252, 0.45172967,
       0.10533261])

Create tf.data.dataset

In [30]:
BUFFER_SIZE = len(context)
BATCH_SIZE = 64

is_train = np.random.uniform(size=(len(target),)) < 0.8 ## Draw samples from a uniform distribution between 0 and 1 , if less than 0.8 we are keeping those records in training

train_raw = (tf.data.Dataset
    .from_tensor_slices((context[is_train], target[is_train]))
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE))


val_raw = (tf.data.Dataset
    .from_tensor_slices((context[~is_train], target[~is_train]))
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE))

In [33]:
## visualise two  examples from one batch of 64
for example_context_strings, example_target_strings in train_raw.take(1):
  print(example_context_strings[:2])
  print()
  print(example_target_strings[:2])

tf.Tensor([b"Let's try it once more." b"I don't speak Japanese."], shape=(2,), dtype=string)

tf.Tensor([b'Essayons encore une fois.' b'Je ne parle pas japonais.'], shape=(2,), dtype=string)


In [34]:
# The tensorflow_text package contains a unicode normalize operation, other operations are using from tf.strings.
def tf_lower_and_split_punct(text):
  # Split accented characters.
  text = tf_text.normalize_utf8(text, 'NFKD') ## type of normalisation -  Characters are decomposed into their compatibility equivalents. For example, the character "Ⓐ" (a circled letter A) is decomposed into the regular letter "A".
  text = tf.strings.lower(text)
  # Keep space, a to z, and select punctuation and remove \u202f
  text = tf.strings.regex_replace(text, '[^ a-z.?!,¿]', '') ## this will take care of \u202f
  # Add spaces around punctuation.
  text = tf.strings.regex_replace(text, '[.?!,¿]', r' \0 ')
  # Strip whitespace.
  text = tf.strings.strip(text)

  text = tf.strings.join(['[START]', text, '[END]'], separator=' ')
  return text

In [38]:
print(example_context_strings[:1].numpy())
print(tf_lower_and_split_punct(example_context_strings[:1]).numpy())

[b"Let's try it once more."]
[b'[START] lets try it once more . [END]']


In [66]:

token_counts = Counter(token for text in context for token in text.split())
most_common_tokens_eng = token_counts.most_common()


In [67]:
len(most_common_tokens_eng), most_common_tokens_eng[:20]

(27393,
 [('I', 48373),
  ('to', 38541),
  ('you', 31479),
  ('the', 28621),
  ('a', 24319),
  ('is', 15375),
  ('Tom', 13053),
  ('of', 11685),
  ('in', 10084),
  ('have', 9101),
  ('was', 8915),
  ('that', 8853),
  ('He', 8809),
  ("I'm", 7866),
  ('for', 7825),
  ("don't", 7470),
  ('do', 7329),
  ('You', 7273),
  ('your', 7226),
  ('be', 6773)])

In [69]:

token_counts = Counter(token for text in target for token in text.split())
most_common_token_french = token_counts.most_common()

In [70]:
len(most_common_token_french),most_common_token_french[:20]

(44918,
 [('de', 36253),
  ('Je', 34973),
  ('?', 28473),
  ('pas', 27005),
  ('que', 24757),
  ('à', 21071),
  ('ne', 18979),
  ('la', 17435),
  ('le', 17168),
  ('Il', 13978),
  ('Tom', 13470),
  ('est', 13073),
  ('vous', 12644),
  ('un', 11960),
  ('a', 11638),
  ('ce', 9580),
  ('en', 8845),
  ('une', 8797),
  ('me', 7651),
  ('je', 7633)])